In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [53]:
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, make_scorer

In [4]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

In [34]:
from sklearn.model_selection import GridSearchCV

In [46]:
class HogFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, color_scheme, orientation, pixels_per_cell, cells_per_block):
        self.color_scheme = color_scheme
        self.orientation = orientation
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
    
    def _hog_feature(self, image):
        return hog(image, orientations=self.orientation, 
                   pixels_per_cell=(self.pixels_per_cell, self.pixels_per_cell), 
                   cells_per_block=(self.cells_per_block, self.cells_per_block), 
                   feature_vector=True)
    
    def fit(self, X, y, **fit_params):
        return self
    
    def transform(self, X):
        X_new = []
        for x in X:
            img = cv2.cvtColor(x, self.color_scheme).astype(np.float32)/255
            rhf = self._hog_feature(img[:, :, 0])
            ghf = self._hog_feature(img[:, :, 1])
            bhf = self._hog_feature(img[:, :, 2])
            X_new.append(np.hstack((rhf, ghf, bhf)))
        return np.array(X_new)
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [8]:
def read(data_dir):
    for img_path in glob.iglob(os.path.join(data_dir, '*', '*.png')):
        yield cv2.imread(img_path)

In [9]:
vehicles = [feature for feature in read(VEHICLES)]

In [10]:
non_vehicles = [feature for feature in read(NON_VEHICLES)]

In [11]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [12]:
len(labels) == len(data)

True

In [13]:
data.shape

(17760, 64, 64, 3)

In [14]:
labels.shape

(17760,)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [16]:
x_train.shape

(14208, 64, 64, 3)

In [47]:
clf = svm.LinearSVC()
scaler = StandardScaler()
extractor = HogFeatureExtractor(cv2.COLOR_BGR2YCrCb, 9, 8, 2)
pipeline = Pipeline([('hog', extractor), ('scaler', scaler), ('svc', clf)])
# pipeline.fit(x_train, y_train)

In [48]:
color_schemes = [cv2.COLOR_BGR2RGB, cv2.COLOR_BGR2HLS, cv2.COLOR_BGR2YCrCb]
orientations = [9, 11, 13]
ppc = [8, 16]
cpb = [2, 3]
param_grid = dict(hog__color_scheme=color_schemes,
                  hog__orientation=orientations, 
                  hog__pixels_per_cell=ppc, 
                  hog__cells_per_block=cpb)

In [54]:
grid = GridSearchCV(pipeline, param_grid=param_grid, 
                    scoring=make_scorer(f1_score))

In [ ]:
grid.fit(x_train, y_train)

In [29]:
acc = pipeline.score(x_test, y_test)

In [30]:
acc

0.98761261261261257

In [31]:
pred = pipeline.predict(x_test)

In [32]:
cm = confusion_matrix(y_test, pred)

In [33]:
cm

array([[1842,   16],
       [  28, 1666]])

In [19]:
with open('classifier.p', 'wb') as _file:
    pickle.dump(pipeline, _file)

In [20]:
with open('classifier.p', 'rb') as _file:
    pipex = pickle.load(_file)

In [21]:
pred = pipex.predict(x_test)
cm = confusion_matrix(y_test, pred)
cm

array([[1775,   28],
       [  43, 1706]])